# NL2SQL Internal Demo
## Running this notebook requires certain permissions, which are automatically granted via the Google Group [g/nl2sql-users](http://g/nl2sql-users). Please ensure you have joined this group before running this noteook.
### This notebook contains internal, non-public code and assets. Please do not share / download anything from here. This project is still under development and might not be suitable for all POC / Production use cases. To enable continuous improvements, this  project is heavily logged, so please be mindful of the information you share here. Please ensure that the data you want to test this on has already been loaded into BigQuery (or any other accessible database) and that the current user has the appropriate database permissions.
#### Get in Touch: [go/NL2SQL](http://go/nl2sql)

In [ ]:
#@title Environment Setup
PROJECT_ID = " " # @param {type:"string"}
# LIB_VERISON = "2177683"
LIB_VERISON = "2238084"


PROJECT_ID = PROJECT_ID.strip()

if not PROJECT_ID:
  raise RuntimeError("Please provide a valid Project ID")

import os
import time
from google.colab import auth
from IPython.display import HTML, display, clear_output
from google.colab import data_table

auth.authenticate_user()
activeaccounts = !gcloud config get-value account
!gcloud config set project $PROJECT_ID
build_dir = 'CONTINUOUS_INTEGRATION' if (
  LIB_VERISON == "latest"
) else f"PRESUBMIT_GERRIT_ON_BORG/{LIB_VERISON}"

builds = !gsutil ls gs://nl2sql-internal/builds/{build_dir}/**
last_build = next(filter(lambda x: x.endswith('.whl'), sorted(builds, reverse=True)))
!gsutil cp {last_build} .
clear_output()
!pip install bigframes==0.25
os.environ["NL2SQL_ENABLE_ANALYTICS"] = "True"
os.environ["NL2SQL_LOG_BUCKET"] = "nl2sql-logs"
!pip install -qq {last_build.split('/')[-1]}

import warnings
import vertexai
from loguru import logger
from sqlalchemy.exc import SAWarning
vertexai.init(project=PROJECT_ID, location="us-central1")
logger.disable("nl2sql.commons.reporting.fingerprint")
warnings.filterwarnings("ignore", category=SAWarning)
!pip install -qq ipython-autotime
%load_ext autotime
clear_output()
display(
  HTML(
    f"""
    <h1><span>Installed NL2SQL Library</span></h1>
    <span>Installed Version : <code>{last_build.split('/', 4)[-1]}</code></span>
    """
  )
)

time: 5.39 ms (started: 2024-03-27 16:44:04 +00:00)


In [ ]:
#@title Dataset Setup

# @markdown If you already have a BQ dataset you want to try with NL2SQL, please provide the name of the dataset below.
# @markdown If you want to try NL2SQL with a dummy dataset, leave this blank and we will create a dummy dataset in the project defined above.

from google.cloud import bigquery
from google.cloud.exceptions import NotFound

client = bigquery.Client()

dataset_name = " " # @param {type:"string"}
dataset_name = dataset_name.strip()
bigquery_connection_string = None
if dataset_name:
  print(f"Checking if dataset {dataset_name} exists in {PROJECT_ID}.")
  try:
    client.get_dataset(f"{PROJECT_ID}.{dataset_name}")
  except NotFound as e:
    raise RuntimeError(
      f"Dataset {dataset_name} does not exist in {PROJECT_ID}. Please provide a"
      f" valid dataset name, or leave the field blank to use a dummy dataset."
    ) from e
  else:
    print("Dataset found, proceeding.")
    bigquery_connection_string = f"bigquery://{PROJECT_ID}/{dataset_name}"
else:
  dataset_name = "libraries_io"
  try:
    client.get_dataset(f"{PROJECT_ID}.{dataset_name}")
    print(
      f"The dummy dataset {dataset_name} is already present in {PROJECT_ID}. "
      f"This dataset will be used henceforth, please ensure it has valid data."
    )
    bigquery_connection_string = f"bigquery://{PROJECT_ID}/{dataset_name}"
  except NotFound as e:
    !gcloud services enable bigquerydatatransfer.googleapis.com
    !bq mk --dataset --location=US {PROJECT_ID}:{dataset_name}
    !bq cp --clone -n bigquery-public-data:libraries_io.dependencies {PROJECT_ID}:libraries_io.dependencies
    !bq cp --clone -n bigquery-public-data:libraries_io.projects {PROJECT_ID}:libraries_io.projects
    !bq cp --clone -n bigquery-public-data:libraries_io.projects_with_repository_fields {PROJECT_ID}:libraries_io.projects_with_repository_fields
    !bq cp --clone -n bigquery-public-data:libraries_io.repositories {PROJECT_ID}:libraries_io.repositories
    !bq cp --clone -n bigquery-public-data:libraries_io.repository_dependencies {PROJECT_ID}:libraries_io.repository_dependencies
    !bq cp --clone -n bigquery-public-data:libraries_io.tags {PROJECT_ID}:libraries_io.tags
    !bq cp --clone -n bigquery-public-data:libraries_io.versions {PROJECT_ID}:libraries_io.versions
    clear_output()
    bigquery_connection_string = f"bigquery://{PROJECT_ID}/libraries_io"
  finally:
    print(
      "\n", "="*25, "\n",
      f"\ndataset_name = \"{dataset_name}\"",
      f"\nbigquery_connection_string = \"{bigquery_connection_string}\"\n"
    )

# Basic Workflow

In [ ]:
#@title Executor Setup

from nl2sql.executors.linear_executor.core import CoreLinearExecutor


executor = CoreLinearExecutor.from_connection_string_map(
    {
        dataset_name: bigquery_connection_string
    }
)
print("\n\n", "="*25, "Executor Created", "="*25, "\n\n")
print("Executor ID :", executor.executor_id)

2024-01-29 12:40:12.486 | INFO     | nl2sql.datasets.base:fetch_schema:268 - [libraries_io] : Fetching Schema ...
2024-01-29 12:40:18.596 | SUCCESS  | nl2sql.datasets.base:fetch_schema:289 - [libraries_io] : Schema Obtained Successfully
2024-01-29 12:40:18.600 | DEBUG    | nl2sql.datasets.base:from_connection_string:303 - [libraries_io] : Analysing ...
2024-01-29 12:40:25.630 | SUCCESS  | nl2sql.datasets.base:from_connection_string:319 - [libraries_io] : Analysis Complete
2024-01-29 12:40:25.632 | DEBUG    | nl2sql.datasets.base:model_post_init:360 - [libraries_io] : Instantiating ...
2024-01-29 12:40:25.636 | DEBUG    | nl2sql.datasets.base:model_post_init:361 - [libraries_io] : Calculating Exclusions ...
2024-01-29 12:40:25.638 | INFO     | nl2sql.datasets.base:model_post_init:375 - [libraries_io] : No tables will be excluded
2024-01-29 12:40:25.641 | SUCCESS  | nl2sql.datasets.base:model_post_init:376 - [libraries_io] : Exclusions Calculated
2024-01-29 12:40:25.644 | DEBUG    | nl2s



 ========================= Executor Created ========================= 


Executor ID : ac2fb792dc254fe28a8a9ac0553751c1
time: 59.6 s (started: 2024-01-29 12:39:58 +00:00)


In [ ]:
#@title SQL Generation

result = executor(
    db_name= dataset_name,
    question = "What is the name of the project with the highest source rank?" # @param {type:"string"}
)
print("\n\n", "="*50, "Generated SQL", "="*50, "\n\n")
print("Result ID:", result.result_id, "\n\n")
print(result.generated_query)

2024-01-29 12:40:58.559 | INFO     | nl2sql.executors.linear_executor.core:__call__:56 - Running Executor.LinearExecutor.CoreLinearExecutor ...
2024-01-29 12:40:58.565 | INFO     | nl2sql.tasks.table_selection.core:__call__:119 - Running Task.TableSelection.CoreTableSelector ...
2024-01-29 12:40:59.203 | DEBUG    | nl2sql.tasks.table_selection.core:__call__:149 - [Task.TableSelection.CoreTableSelector] : Received LLM Response : {"generations": [[{"text": " projects", "generation_info": {"is_blocked": false, "safety_attributes": {"Finance": 0.3, "Insult": 0.1, "Sexual": 0.1}}, "type": "Generation"}]], "llm_output": null, "run": [{"run_id": "de104c92-ea59-4056-b2ee-2f5e84f2306d"}]}
2024-01-29 12:40:59.208 | DEBUG    | nl2sql.datasets.base:model_post_init:360 - [libraries_io] : Instantiating ...
2024-01-29 12:40:59.211 | DEBUG    | nl2sql.datasets.base:model_post_init:361 - [libraries_io] : Calculating Exclusions ...
2024-01-29 12:40:59.215 | INFO     | nl2sql.datasets.base:model_post_ini



 ================================================== Generated SQL ================================================== 


Result ID: f74302cf2b3b45a4badad2e087894d2f 



SELECT name
FROM projects
ORDER BY sourcerank DESC
LIMIT 1;

time: 2min 1s (started: 2024-01-29 12:40:58 +00:00)


In [ ]:
#@title SQL Execution

df = executor.fetch_result(result)
data_table.DataTable(df)

,name
0,react


time: 1.88 s (started: 2024-01-29 12:42:59 +00:00)


# Using Few Shot Chain of Thoughts with a Larger Context model

In [ ]:
#@title Executor Setup

from nl2sql.llms.vertexai import text_bison_32k
from nl2sql.executors.linear_executor.core import CoreLinearExecutor
from nl2sql.tasks.table_selection.core import CoreTableSelector, prompts as cts_prompts
from nl2sql.tasks.column_selection.core import CoreColumnSelector, prompts as ccs_prompts
from nl2sql.tasks.sql_generation.core import CoreSqlGenerator, prompts as csg_prompts

llm = text_bison_32k()

# Disabling logs because these steps generate a LOT of logs.
logger.disable("nl2sql.datasets.base")
core_table_selector = CoreTableSelector(llm=llm, prompt=cts_prompts.CURATED_FEW_SHOT_COT_PROMPT)
core_column_selector = CoreColumnSelector(llm=llm, prompt=ccs_prompts.CURATED_FEW_SHOT_COT_PROMPT)
core_sql_generator = CoreSqlGenerator(llm=llm, prompt=csg_prompts.CURATED_FEW_SHOT_COT_PROMPT)
logger.enable("nl2sql.datasets.base")

cot_executor = CoreLinearExecutor.from_connection_string_map(
    {
        dataset_name: bigquery_connection_string
    },
    core_table_selector = core_table_selector,
    core_column_selector = core_column_selector,
    core_sql_generator = core_sql_generator
)

2024-01-29 13:06:22.882 | INFO     | nl2sql.assets.prompts:TASK_TABLE_SELECTION_CORE_V1_SPIDER_V1:215 - Instantiating TASK_TABLE_SELECTION_CORE_V1_SPIDER_V1
2024-01-29 13:06:36.870 | INFO     | nl2sql.assets.prompts:TASK_COLUMN_SELECTION_CORE_V1_SPIDER_V1:224 - Instantiating TASK_COLUMN_SELECTION_CORE_V1_SPIDER_V1
2024-01-29 13:06:47.692 | INFO     | nl2sql.assets.prompts:TASK_SQL_GENERATION_CORE_V1_SPIDER_V1:242 - Instantiating TASK_SQL_GENERATION_CORE_V1_SPIDER_V1
2024-01-29 13:06:52.528 | INFO     | nl2sql.datasets.base:fetch_schema:268 - [libraries_io] : Fetching Schema ...
2024-01-29 13:06:58.660 | SUCCESS  | nl2sql.datasets.base:fetch_schema:289 - [libraries_io] : Schema Obtained Successfully
2024-01-29 13:06:58.663 | DEBUG    | nl2sql.datasets.base:from_connection_string:303 - [libraries_io] : Analysing ...
2024-01-29 13:07:06.340 | SUCCESS  | nl2sql.datasets.base:from_connection_string:319 - [libraries_io] : Analysis Complete
2024-01-29 13:07:06.347 | DEBUG    | nl2sql.datasets

time: 1min 17s (started: 2024-01-29 13:06:22 +00:00)


In [ ]:
#@title SQL Generation

cot_result = cot_executor(
    db_name= dataset_name,
    question = "Top 10 individual projects with the highest number of runtime dependencies" # @param {type:"string"}
)
print("\n\n", "="*50, "Generated SQL", "="*50, "\n\n")
print("Result ID:", cot_result.result_id, "\n\n")
print(cot_result.generated_query)

2024-01-29 13:16:03.969 | INFO     | nl2sql.executors.linear_executor.core:__call__:56 - Running Executor.LinearExecutor.CoreLinearExecutor ...
2024-01-29 13:16:03.975 | INFO     | nl2sql.tasks.table_selection.core:__call__:119 - Running Task.TableSelection.CoreTableSelector ...
2024-01-29 13:16:07.110 | DEBUG    | nl2sql.tasks.table_selection.core:__call__:149 - [Task.TableSelection.CoreTableSelector] : Received LLM Response : {"generations": [[{"text": " The question mentions \"projects\", and the table contains information about projects. Yes. The question mentions \"runtime dependencies\", and the table contains information about dependencies and their types in the column \"dependency_kind\". Yes. The question asks about the number of dependencies, and the table contains information about dependencies. Yes. The table contains information about projects and their dependencies, and the question also asks about projects with a given number of dependencies. Yes. Final Answer : Yes", "g



 ================================================== Generated SQL ================================================== 


Result ID: a2ddffd76b784538921fbe3e52514d3e 


SELECT
  project_name,
  COUNT(*) AS dependency_count
FROM
  dependencies
WHERE
  dependency_kind = 'runtime'
GROUP BY
  project_name
ORDER BY
  dependency_count DESC
LIMIT 10
time: 54.1 s (started: 2024-01-29 13:16:03 +00:00)


In [ ]:
#@title SQL Execution

cot_df = cot_executor.fetch_result(cot_result)
data_table.DataTable(cot_df)

,project_name,dependency_count
0,bpmn-studio,41670
1,nuxt-edge,35403
2,arachne-ui,32923
3,react-misc-toolbox,28150
4,pri,26241
5,prisma-cli-engine,25556
6,@atomist/sdm,18685
7,prisma-cli-core,17397
8,wm-cli,15912
9,mwp-consumer,15749


time: 1.63 s (started: 2024-01-29 13:17:13 +00:00)


# Excel File
## Using Sample Data from Excel File.

[Example File](gs://nl2sql-internal/assets/datasets/custom/custom-dataset.xlsx)

- Each Sheet in this file Represents a sample table with sheetname representing the Table name.
- Each Sheet has some sample rows/example records.
- There are two special Sheets in the Excel File i.e.
    - **Primary Key** : Key Value Style Representation where First Column i.e. "*Table*" is Tablename/Sheetname and Second Column i.e. "*Primary Key*" is a comma seprated list of columns representing Primary Key in that table.
    - **Foreign Key** : Contains Three Columns where First Column i.e. "*Table*" is Tablename/Sheetname and Second Column i.e. "*Foreign Key* " is a single value representing the Foreign Key Column in the associated Table. The Third Column i.e. "*References*" is the Reference Column where each value is of type "*Reference Table (Reference Key Column)*" to represent the referential integrity between the table relationships

In [ ]:
#@title Download the Sample Excel File in colab
!gsutil cp  gs://nl2sql-internal/assets/datasets/custom/custom-dataset.xlsx .

Copying gs://nl2sql-internal/assets/datasets/custom/custom-dataset.xlsx...
/ [1 files][166.4 KiB/166.4 KiB]                                                
Operation completed over 1 objects/166.4 KiB.                                    
time: 2.82 s (started: 2024-01-29 12:45:49 +00:00)


In [ ]:
#@title Executor Setup

from nl2sql.llms.vertexai import text_bison_32k
from nl2sql.executors.linear_executor.core import CoreLinearExecutor

llm = text_bison_32k()

excel_filepath = "/content/custom-dataset.xlsx" # @param {type:"string"}
gcp_project_id = "gdc-ai-playground" # @param {type:"string"}
dataset_name = "custom_dataset" # @param {type:"string"}

excel_file_executor = CoreLinearExecutor.from_excel(
                            filepath=excel_filepath,
                            project_id=gcp_project_id,
                            dataset_name=dataset_name
                        )



2024-01-29 12:45:53.540 | INFO     | nl2sql.datasets.custom:from_excel:116 - Dataset gdc-ai-playground.custom_dataset already exists.
2024-01-29 12:45:58.768 | SUCCESS  | nl2sql.datasets.custom:create_tables:197 - Created table custom_dataset.customers
2024-01-29 12:46:01.784 | SUCCESS  | nl2sql.datasets.custom:create_tables:197 - Created table custom_dataset.employees
2024-01-29 12:46:05.463 | SUCCESS  | nl2sql.datasets.custom:create_tables:197 - Created table custom_dataset.offices
2024-01-29 12:46:08.681 | SUCCESS  | nl2sql.datasets.custom:create_tables:197 - Created table custom_dataset.orderdetails
2024-01-29 12:46:11.892 | SUCCESS  | nl2sql.datasets.custom:create_tables:197 - Created table custom_dataset.orders
2024-01-29 12:46:15.010 | SUCCESS  | nl2sql.datasets.custom:create_tables:197 - Created table custom_dataset.payments
2024-01-29 12:46:17.926 | SUCCESS  | nl2sql.datasets.custom:create_tables:197 - Created table custom_dataset.productlines
2024-01-29 12:46:21.580 | SUCCESS

time: 1min 29s (started: 2024-01-29 12:45:52 +00:00)


In [ ]:
#@title SQL Generation

excel_file_result = excel_file_executor(
    db_name=dataset_name,
    question="What is the avg order price?" # @param {type:"string"}
)

print("\n\n", "="*50, "Generated SQL", "="*50, "\n\n")
print("Result ID:", excel_file_result.result_id, "\n\n")
print(excel_file_result.generated_query)

2024-01-29 12:47:22.681 | INFO     | nl2sql.executors.linear_executor.core:__call__:56 - Running Executor.LinearExecutor.CoreLinearExecutor ...
2024-01-29 12:47:22.685 | INFO     | nl2sql.tasks.table_selection.core:__call__:119 - Running Task.TableSelection.CoreTableSelector ...
2024-01-29 12:47:23.362 | DEBUG    | nl2sql.tasks.table_selection.core:__call__:149 - [Task.TableSelection.CoreTableSelector] : Received LLM Response : {"generations": [[{"text": " orders, orderdetails", "generation_info": {"is_blocked": false, "safety_attributes": {"Finance": 0.7, "Health": 0.2, "Insult": 0.1, "Sexual": 0.1}}, "type": "Generation"}]], "llm_output": null, "run": [{"run_id": "e7df45cf-5438-47f8-9e01-8014bc866801"}]}
2024-01-29 12:47:23.365 | DEBUG    | nl2sql.datasets.base:model_post_init:360 - [custom_dataset] : Instantiating ...
2024-01-29 12:47:23.367 | DEBUG    | nl2sql.datasets.base:model_post_init:361 - [custom_dataset] : Calculating Exclusions ...
2024-01-29 12:47:23.371 | INFO     | nl2s



 ================================================== Generated SQL ================================================== 


Result ID: 5866a7ec4d664e8f8eeefe25e4606537 


SELECT AVG(priceEach) AS average_order_price FROM orderdetails;
time: 17.4 s (started: 2024-01-29 12:47:22 +00:00)


In [ ]:
#@title SQL Execution
df = excel_file_executor.fetch_result(excel_file_result)
data_table.DataTable(df)

,average_order_price
0,90.769499


time: 1.65 s (started: 2024-01-29 12:47:40 +00:00)


# Disabling Table and Column Selection, and using ReAct for SQL Generation with a Larger Context model (Experimental)

In [ ]:
#@title Executor Setup

from nl2sql.tasks.sql_generation.react import ReactSqlGenerator

react_executor = CoreLinearExecutor.from_connection_string_map(
    {
        dataset_name: bigquery_connection_string
    },
    core_table_selector = None,
    core_column_selector = None,
    core_sql_generator = ReactSqlGenerator(llm=llm)
)

2024-01-29 12:47:42.361 | INFO     | nl2sql.datasets.base:fetch_schema:268 - [libraries_io] : Fetching Schema ...
2024-01-29 12:47:48.348 | SUCCESS  | nl2sql.datasets.base:fetch_schema:289 - [libraries_io] : Schema Obtained Successfully
2024-01-29 12:47:48.351 | DEBUG    | nl2sql.datasets.base:from_connection_string:303 - [libraries_io] : Analysing ...
2024-01-29 12:47:54.415 | SUCCESS  | nl2sql.datasets.base:from_connection_string:319 - [libraries_io] : Analysis Complete
2024-01-29 12:47:54.418 | DEBUG    | nl2sql.datasets.base:model_post_init:360 - [libraries_io] : Instantiating ...
2024-01-29 12:47:54.419 | DEBUG    | nl2sql.datasets.base:model_post_init:361 - [libraries_io] : Calculating Exclusions ...
2024-01-29 12:47:54.421 | INFO     | nl2sql.datasets.base:model_post_init:375 - [libraries_io] : No tables will be excluded
2024-01-29 12:47:54.424 | SUCCESS  | nl2sql.datasets.base:model_post_init:376 - [libraries_io] : Exclusions Calculated
2024-01-29 12:47:54.426 | DEBUG    | nl2s

time: 40.6 s (started: 2024-01-29 12:47:41 +00:00)


In [ ]:
#@title SQL Generation

react_result = react_executor(
    db_name= dataset_name,
    question = "Top 10 projects with the highest number of published versions and the stars count of their repositories" # @param {type:"string"}
)
print("\n\n", "="*50, "Generated SQL", "="*50, "\n\n")
print("Result ID:", react_result.result_id, "\n\n")
print(react_result.generated_query)

2024-01-29 12:50:58.316 | INFO     | nl2sql.executors.linear_executor.core:__call__:56 - Running Executor.LinearExecutor.CoreLinearExecutor ...
2024-01-29 12:50:58.320 | INFO     | nl2sql.tasks.sql_generation.react:__call__:46 - Running Task.SqlGeneration.ReactSqlGenerator ...




 ================================================== Generated SQL ================================================== 


Result ID: fd2415514bb24ca291af8afb17b2c5e5 


SELECT p.name, p.versions_count, r.stars_count
FROM projects p
LEFT JOIN repositories r ON p.repository_id = r.id
ORDER BY p.versions_count DESC, r.stars_count DESC
LIMIT 10
time: 2min 22s (started: 2024-01-29 12:50:58 +00:00)


In [ ]:
#@title SQL Execution

react_df = react_executor.fetch_result(react_result)
data_table.DataTable(react_df)

,name,versions_count,stars_count
0,Momentum.Pm.PortalApi,6442,NaN
1,Momentum.Pm.Api,5757,NaN
2,lookout-artifactory-test,5301,NaN
3,electron-apps,5121,NaN
4,ccxt,4445,9567.0
5,ccxt,4426,9567.0
6,wix-style-react,4407,NaN
7,botfather,4204,NaN
8,owner-profiles,3888,NaN
9,vui-ad-hoc-alexa-recognizer,3664,NaN


time: 1.92 s (started: 2024-01-29 13:06:11 +00:00)
